In [ ]:
# This notebook is based on the course offered on Udemy
# https://www.udemy.com/course/tensorflow-developer-certificate-machine-learning-zero-to-mastery/

In [3]:
import zipfile

zip_ref = zipfile.ZipFile("pizza_steak.zip")
zip_ref.extractall()
zip_ref.close()

In [ ]:
# Unzip the downloaded file 
